### 1. A positive and negative word list (Hu and Bing 2004) exists on CourseWorks (Files à HW_FILEs à HW3 àzip). Please create a function called gen_senti that Tokenizes arbitrary text and compares each token with the positive and negative lexicons of each dictionary and outputs the sentiment score, S.  Positive and negative words, pw and nw, count as a score of 1 and -1 respectively for each word matched.  The total count for pw and nw are pc and nc, respectively.  Each message sentiment, S, is normalized between -1 and 1.  Any text that does not any positive AND negative words would have to be ignored, and not scored. (60 points)

For example: Let us say the following sentence was an input into the function “The darkest hour is among us in this time of gloom, however, we will prevail!”.  Let us say the negative words were darkest and gloom and positive words were prevail

S = (-1 + -1 + 1) / 3 = -1/3 = -0.3333

In [17]:
%pwd

'D:\\QMSS\\Spring\\NaturalLanguageProcessing\\Homework\\HW3'

In [16]:
%cd "D:\QMSS\Spring\NaturalLanguageProcessing\Homework\HW3"

D:\QMSS\Spring\NaturalLanguageProcessing\Homework\HW3


#### To make it so that the example provided could be tested the word darkest was added to the negative word lexicon and prevail was added to the positive word lexicon. Gloom was already within the negative word lexicon, so it did not require being added for the example provided to make sense.  

In [3]:
with open("negative-words.txt", "r") as f:
    negText = f.read()
negTokens = negText.split("\n") # This splits the text file into tokens on the new line character
print(negTokens[-10:])

['wrought', 'yawn', 'zap', 'zapped', 'zaps', 'zealot', 'zealous', 'zealously', 'zombie', 'darkest']


In [4]:
with open("positive-words.txt", "r") as f:
    posText = f.read()
posTokens = posText.split("\n") # This splits the text file into tokens on the new line character
print(posTokens[-10:])

['wowed', 'wowing', 'wows', 'yay', 'youthful', 'zeal', 'zenith', 'zest', 'zippy', 'prevail']


In [5]:
the_sent = "The darkest hour is among us in this time of gloom, however, we will prevail!"

In [6]:
import re
from nltk.tokenize import word_tokenize


def gen_senti(theText):
    # Count positive words
    numPosWords = 0
    theTextTokens = word_tokenize(theText)
    for word in theTextTokens:
        if word in posTokens:
            numPosWords += 1
            
    # Count negative words
    numNegWords = 0
    for word in theTextTokens:
        if word in negTokens:
            numNegWords += -1
    
    
    
    sum = (numPosWords + numNegWords)/(abs(numPosWords)+abs(numNegWords))
    return sum


In [7]:
gen_senti(the_sent) #The sentence would deemed generally negative 

-0.3333333333333333

### Code exists on CourseWorks (Files à HW_FILEs à HW3 àpy and crawler.py). Only run the crawler function once with the exact words you see for the_query = [‘positive words’, ‘negative words’].  Note: you only need to run fetch_crawl once, comment that part out after running it

1. What is the function iterate_var? (10 points)
2. What is the function grid_search_func? (10 points)
3. For the exact sentence above “The darkest…” what sentiment score do you get from the function above? (5 points)
4. The variable called the_result outputs the likelihood score for the text entered in my_sample as a negative_word or a positive_word. Meaning if the_result has scores of 0.75 and 0.25 for negative_word and positive_word, respectively, which means the_result is more likely to be a negative_word.  Run the code, what scores do you get for the the_result? (5 points)
5. Change the ngram_range=(1,1) to ngram_range=(1,3), what scores do you get for the the_result and what observations can you make on why scores changed between uni-gram (1,1) and tri-gram(1,3)? (10 points)
***Upload a zip file with your .py code and comments in code to answer questions above***

#### 1.  The iterate_var function makes it so that Principal Component analysis can be performed on our tf-idf vectorized data frame. This function allows one to set a limit of how close to one we want the percentage of variance explained by each of the selected components to be allowed to get, as the var_fig, which is equal to the sume of explained ratios, is less than the target provided to the function. In the example provided PCA will only be applied when the sum of the explained variance ratios is less than or equal to the target variance, provided to the iterate_var function.

In [ ]:
from sklearn.decomposition import PCA

def iterate_var(var_target):
    var_fig = 0.0
    cnt = 1
    while var_fig <= var_target:
        pca = PCA(n_components=cnt)
        my_dim = pca.fit_transform(my_xform_tfidf)
        var_fig = sum(pca.explained_variance_ratio_)   
        cnt += 1
    return my_dim, pca

my_dim, pca = iterate_var(0.95)


#### 2.  The grid_search_func applies gird search CV to the model and saves the best model, the best score, and the best parameters. This function allows us to proceed forward knowing that we fitting the model that will predict at the highest level of accuracy, provided we set effective parameters for GridSearchCV to leverage. 

In [ ]:
def grid_search_func(param_grid, the_mode_in, the_vec_in, the_lab_in):
    grid_search = GridSearchCV(the_mode_in, param_grid=param_grid, cv=5)
    best_model = grid_search.fit(the_vec_in, the_lab_in)
    max_score = grid_search.best_score_
    best_params = grid_search.best_params_

    return best_model, max_score, best_params

param_grid = {"max_depth": [10, 50, 100],
              "n_estimators": [1, 4, 16, 32, 64]}


#### 3. We get a primiarily negative score, with the sentence containing 58.59% negative words and 41.4123% positive words.

In [10]:
import pandas as pd

In [23]:
from crawlerHw3 import crawler_sol
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
from sklearn.model_selection import GridSearchCV

#array that contains what topics to crawl
the_query = ['positive words', 'negative words']
#number of document to crawl per topic
num_docs = 100

#initialize function
my_func = crawler_sol()

#call up crawler function and perform crawl, resultant dataframe contains 3 columns, basic wranged
#data
the_data = my_func.fetch_crawl(None, the_query, num_docs)

#####
my_vec = CountVectorizer(ngram_range=(1, 1))

my_xform_vec = my_vec.fit_transform(the_data.body_basic).toarray()
col_names = my_vec.get_feature_names()
#count_list = dict(my_xform_vec.sum(axis=0)) #this gets word frequency counts into a dictionary

my_xform_vec = pd.DataFrame(my_xform_vec, columns=col_names)
#####

#####
my_vec_tfidf = TfidfVectorizer()

my_xform_tfidf = my_vec_tfidf.fit_transform(the_data.body_basic).toarray()
col_names = my_vec_tfidf.get_feature_names()

my_xform_tfidf = pd.DataFrame(my_xform_tfidf, columns=col_names)

#####
#
#clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
#
#clf.fit(my_xform_tfidf, the_data.label)  
#
#my_sample = ['i hiked in the woods and ran into a bobcat']
#test_text = my_vec_tfidf.transform(my_sample).toarray()
#
#clf.predict(test_text)

#####

from sklearn.decomposition import PCA

def iterate_var(var_target):
    var_fig = 0.0
    cnt = 1
    while var_fig <= var_target:
        pca = PCA(n_components=cnt)
        my_dim = pca.fit_transform(my_xform_tfidf)
        var_fig = sum(pca.explained_variance_ratio_)   
        cnt += 1
    return my_dim, pca

my_dim, pca = iterate_var(0.95)

def grid_search_func(param_grid, the_mode_in, the_vec_in, the_lab_in):
    grid_search = GridSearchCV(the_mode_in, param_grid=param_grid, cv=5)
    best_model = grid_search.fit(the_vec_in, the_lab_in)
    max_score = grid_search.best_score_
    best_params = grid_search.best_params_

    return best_model, max_score, best_params

param_grid = {"max_depth": [10, 50, 100],
              "n_estimators": [1, 4, 16, 32, 64]}

clf_pca = RandomForestClassifier()
gridsearch_model, best, opt_params = grid_search_func(
        param_grid, clf_pca, my_xform_tfidf, the_data.label)

clf_pca = RandomForestClassifier()
clf_pca.set_params(**gridsearch_model.best_params_)
clf_pca.fit(my_dim, the_data.label)

my_sample = ['The darkest hour is among us in this time of gloom, however, we will prevail!']
test_text = my_vec_tfidf.transform(my_sample).toarray()
test_text_pca = pca.transform(test_text)

the_result = pd.DataFrame(clf_pca.predict_proba(test_text_pca))
the_result.columns = clf_pca.classes_
print (the_result)

   negative_words  positive_words
0        0.585877        0.414123


D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


#### 4. The variable called the_result outputs the likelihood score for the text entered in my_sample as a negative_word or a positive_word. Meaning if the_result has scores of 0.75 and 0.25 for negative_word and positive_word, respectively, which means the_result is more likely to be a negative_word. Run the code, what scores do you get for the the_result? (5 points)

In [24]:
the_result

,negative_words,positive_words
0,0.585877,0.414123


The scores we get are 0.585877 for negative words and 0.414123 for positive words

#### 5.

The original running of the model would only leverage unigrams (1,1). However with the switch to (1,3) the model can now leverage unigrams and trigrams. 

The scores may have changed because the ngram represents the 'size' of the sequence of n terms in the given sample. A 1-gram word sequence would base scores on data being introduced as such: the,darkest,hour,is,among,us,etc. A 3-gram of trigram would introduce data in the following manner: the darkest hour, is among us, etc etc etc. 

The decrease in negative word representation and increase in positive word proportions may have been inspired from shrinking the data size, as a result of including both unigrams and trigrams. The only unigram sentance would take each word as its own and thus the size of the data was replicative of the length of the sentance. One trigrams became leverageable it allowed for the data set to shrink as now the groupings could contain 1 or 3 words of the 15 total. 

If we were to have only one trigram present and the resest unigram the size of the data set would shrink from 15 to 13 as the one trigram takes three of the orginal words and replaces them with one combination of all three words. This could have made it so that the positive words not made up a greater proportion of the sentance, and thus experienced an increase in how represented positive words were in the sentance. 

In [25]:
#array that contains what topics to crawl
the_query = ['positive words', 'negative words']
#number of document to crawl per topic
num_docs = 100

#initialize function
my_func = crawler_sol()

#call up crawler function and perform crawl, resultant dataframe contains 3 columns, basic wranged
#data
the_data = my_func.fetch_crawl(None, the_query, num_docs)

#####
my_vec = CountVectorizer(ngram_range=(1, 3)) #size of the number of in this case words that can be in each token

my_xform_vec = my_vec.fit_transform(the_data.body_basic).toarray()
col_names = my_vec.get_feature_names()
#count_list = dict(my_xform_vec.sum(axis=0)) #this gets word frequency counts into a dictionary

my_xform_vec = pd.DataFrame(my_xform_vec, columns=col_names)
#####

#####
my_vec_tfidf = TfidfVectorizer()

my_xform_tfidf = my_vec_tfidf.fit_transform(the_data.body_basic).toarray()
col_names = my_vec_tfidf.get_feature_names()

my_xform_tfidf = pd.DataFrame(my_xform_tfidf, columns=col_names)

#####
#
#clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
#
#clf.fit(my_xform_tfidf, the_data.label)  
#
#my_sample = ['i hiked in the woods and ran into a bobcat']
#test_text = my_vec_tfidf.transform(my_sample).toarray()
#
#clf.predict(test_text)

#####

from sklearn.decomposition import PCA #dimension reduction 

def iterate_var(var_target):
    var_fig = 0.0
    cnt = 1
    while var_fig <= var_target:
        pca = PCA(n_components=cnt)
        my_dim = pca.fit_transform(my_xform_tfidf)
        var_fig = sum(pca.explained_variance_ratio_)   
        cnt += 1
    return my_dim, pca

my_dim, pca = iterate_var(0.95)

def grid_search_func(param_grid, the_mode_in, the_vec_in, the_lab_in):
    grid_search = GridSearchCV(the_mode_in, param_grid=param_grid, cv=5)
    best_model = grid_search.fit(the_vec_in, the_lab_in)
    max_score = grid_search.best_score_
    best_params = grid_search.best_params_

    return best_model, max_score, best_params

param_grid = {"max_depth": [10, 50, 100],
              "n_estimators": [1, 4, 16, 32, 64]}

clf_pca = RandomForestClassifier()
gridsearch_model, best, opt_params = grid_search_func(
        param_grid, clf_pca, my_xform_tfidf, the_data.label)

clf_pca = RandomForestClassifier()
clf_pca.set_params(**gridsearch_model.best_params_)
clf_pca.fit(my_dim, the_data.label)

my_sample = ['The darkest hour is among us in this time of gloom, however, we will prevail!']
test_text = my_vec_tfidf.transform(my_sample).toarray()
test_text_pca = pca.transform(test_text)

the_result = pd.DataFrame(clf_pca.predict_proba(test_text_pca))
the_result.columns = clf_pca.classes_
print (the_result)

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


   negative_words  positive_words
0        0.541667        0.458333
